# Empirical Project 6

---
**Download the code**

To download the code used in this project as a notebook that can be run in Visual Studio Code or Google Colab, right click [here]() and select 'Save Link As', then save it as a `.ipynb` file.

Don’t forget to also download the data into your working directory by following the steps in this project.

---

## Getting started in Python

For this project, you will need the following packages:

- **pandas** for data analysis
- **matplotlib** for data visualisation
- **numpy** for numerical methods

You'll also be using the **warnings** and **pathblib** packages, but these come built-in with Python.

Remember, you can install packages in Visual Studio Code's integrated terminal (click "View > Terminal") by running `conda install packagename` (if using the Anaconda distribution of Python) or `pip install packagename` if not.

Once you have the Python packages installed, you will need to import them into your Python session—and configure any other initial settings.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import warnings

# Set the plot style for prettier charts:
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
plt.rcParams["figure.figsize"] = [6, 3]
plt.rcParams["figure.dpi"] = 150

# Ignore warnings to make nice output
warnings.simplefilter("ignore")

## TODO

download the zip and extract the files .. and .. to data/ within your working directory

## Python Walkthrough 6.1

**Importing data into Python and creating tables and charts**

Before opening an Excel or csv file using Python, you can open the file in spreadsheet software (such as Excel) to understand how it's structured. From looking at the file, we learn that:

* the variable names are in the first row (no need to use the `skiprows` keyword argument)
* missing values are represented by empty cells
* the last variable is in Column S, with short variable descriptions in Column U: it is easier to import everything first and remove the unnecessary data afterwards.

We will call our imported data `df`.

In [ ]:
df = pd.read_csv(Path("data/AMP_graph_manufacturing.csv"))
df.info()

You can see that the penultimate column with values, Column T, was imported as `"Unnamed: 19"` and only contains `NaN`s. The final column of values has been imported into **pandas** comes from Column U in the spreadsheet and contains information about the variables (named `"storage display value"`).

Let's extract the information about the variables in a new **pandas** series called `man_varinfo` and then remove both of these columns from the dataset. To make it easier to see the `man_varinfo`, we'll temporarily override **pandas** column width limits.

In [ ]:
man_varinfo = df.iloc[:, -1].dropna()

with pd.option_context('display.max_colwidth', 80):
    print(man_varinfo)

And now to drop the last two columns:

In [ ]:
df = df.iloc[:, :-2]
df.head()

A few of the variables that have been imported as numbers are actually categorical variables; there are `"mne_f"` , `"mne_d"`, and `"competition2004"`. **pandas** doesn't automatically know what datatypes different variables should have. However, we can set the type of these variables as categorical and we can use labels to define what each of the numbers in the variables represents.

The first two have quite clear labels. For the third, we'll use some string manipulation tools to grab the labels directly from the `man_varinfo` variable.

In [ ]:
lab_mne_f = ["No MNE_f", "MNE_f"]
lab_mne_d = ["No MNE_d", "MNE_d"]
lab_comp2004 = man_varinfo.iloc[16].split("  ")[-1].split(",")
print(lab_comp2004)

Let's now encode these variables as categoricals, with suitable names.

In [ ]:
for col in ["mne_f", "mne_d", "competition2004"]:
    df[col] = pd.Categorical(df[col])

df["mne_f"] = df["mne_f"].cat.rename_categories(lab_mne_f)
df["mne_d"] = df["mne_d"].cat.rename_categories(lab_mne_d)
df["competition2004"] = df["competition2004"].cat.rename_categories(lab_comp2004)

When you create new labels, check that they have been attached to the correct entries. Although we set them as a list here using, for example, `["No MNE_f", "MNE_f"]`, we could have also used a dictionary, for example `{0: "No MNE_f", 1: "MNE_f"}`, mapping old values into new.

To create the tables, we can use a technique called method chaining. Rather than have a series of separate assignment commands using `=`, method chaining "chains" together a series of methods (these are preceeded by `.`). This approach has pros and cons: it can be easier to read, but harder to debug in case of errors. 

First, we will group data by country (using `.groupby`), then calculate the required aggregate statistics for each of these groups (using `.agg`), then order the countries according to their overall score (highest to lowest) (`.sort_values`).

`.groupby` groups the data according to a given column.

`.agg` aggregates data, and returns it with a different index. In combination with `.groupby`, it will return an index based on what column(s) was/were passed to the groupby operation. There are many ways to use `.agg`, including just setting `.agg.mean()` and other functions such as `count()`, `median()`, `sum()`, and `std()`. However, to change the column names of the returned series, you can either pass a dictionary, say `{"columnname": "mean"}`, or pass an object called a tuple that species the new column name, old column name, and the aggregation function, for example `newcolumnname = ("columnname", "count")`.

In [ ]:
table_mean = (
    df
    .groupby("country")
    .agg(
        obs=("management", "count"),
        m_overall=("management", "mean"),
        m_monitor=("monitor", "mean"),
        m_target=("target", "mean"),
        m_incentives=("people", "mean"),
    )
    .sort_values(by="m_overall", ascending=False)
)

table_mean.round(2)

Let’s make the table showing the ranks. We can use the `.agg` function again, but in this case it keeps the same index as before because we're not using a groupby.

We will also drop the `"obs"` column, and rename all of the columns so that they begin with `"r"` (for rank) rather than `"m"`

In [ ]:
table_mean.agg("rank", ascending=False).drop("obs", axis=1)